In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# ------------------------------
# Custom morphological functions
# ------------------------------

def custom_erode(img, kernel):
    pad_h, pad_w = kernel.shape[0]//2, kernel.shape[1]//2
    padded = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=255)
    output = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            region = padded[i:i+kernel.shape[0], j:j+kernel.shape[1]]
            output[i, j] = np.min(region[kernel == 1])
    return output

def custom_dilate(img, kernel):
    pad_h, pad_w = kernel.shape[0]//2, kernel.shape[1]//2
    padded = np.pad(img, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    output = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            region = padded[i:i+kernel.shape[0], j:j+kernel.shape[1]]
            output[i, j] = np.max(region[kernel == 1])
    return output

def custom_open(img, kernel):
    return custom_dilate(custom_erode(img, kernel), kernel)

def custom_close(img, kernel):
    return custom_erode(custom_dilate(img, kernel), kernel)

def custom_tophat(img, kernel):
    return cv2.subtract(img, custom_open(img, kernel))

def custom_blackhat(img, kernel):
    return cv2.subtract(custom_close(img, kernel), img)


# ------------------------------
# Structuring elements
# ------------------------------

def diamond_kernel(size=5):
    assert size % 2 == 1, "Kernel size must be odd"
    k = np.zeros((size, size), dtype=np.uint8)
    mid = size // 2
    for i in range(size):
        for j in range(size):
            if abs(i - mid) + abs(j - mid) <= mid:
                k[i, j] = 1
    return k

kernel_types = {
    "Rectangular": cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)),
    "Elliptical": cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)),
    "Cross": cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5)),
    "Diamond": diamond_kernel(5)
}

# ------------------------------
# Morphological operations
# ------------------------------

builtin_ops = {
    "Erosion": lambda i, k: cv2.erode(i, k),
    "Dilation": lambda i, k: cv2.dilate(i, k),
    "Opening": lambda i, k: cv2.morphologyEx(i, cv2.MORPH_OPEN, k),
    "Closing": lambda i, k: cv2.morphologyEx(i, cv2.MORPH_CLOSE, k),
    "Top-hat": lambda i, k: cv2.morphologyEx(i, cv2.MORPH_TOPHAT, k),
    "Black-hat": lambda i, k: cv2.morphologyEx(i, cv2.MORPH_BLACKHAT, k)
}

custom_ops = {
    "Erosion": custom_erode,
    "Dilation": custom_dilate,
    "Opening": custom_open,
    "Closing": custom_close,
    "Top-hat": custom_tophat,
    "Black-hat": custom_blackhat
}

# ------------------------------
# Load image
# ------------------------------

img_path = "../images/hist_source.jpg"
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError(f"Image not found at {img_path}")

# ------------------------------
# Create output folder
# ------------------------------

output_root = "morphological_results"
os.makedirs(output_root, exist_ok=True)

# ------------------------------
# Perform and save each operation
# ------------------------------

for op_name, builtin_func in builtin_ops.items():
    print(f"Processing {op_name}...")

    # Create folder for this operation
    op_folder = os.path.join(output_root, op_name)
    os.makedirs(op_folder, exist_ok=True)

    custom_func = custom_ops[op_name]

    for k_name, k in kernel_types.items():
        # Built-in result
        built_result = builtin_func(img, k)
        built_path = os.path.join(op_folder, f"{k_name}_OpenCV.png")
        cv2.imwrite(built_path, built_result)

        # Custom result
        cust_result = custom_func(img, k)
        cust_path = os.path.join(op_folder, f"{k_name}_Custom.png")
        cv2.imwrite(cust_path, cust_result)

print("\n✅ All results saved in:", os.path.abspath(output_root))


Processing Erosion...
Processing Dilation...
Processing Opening...
Processing Closing...


KeyboardInterrupt: 